In [1]:
import time
import sys
sys.path.append('../')
import librairies.dagfeaturingfx 
from  librairies.dagfeaturingfx import *
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import joblib
from joblib import Parallel,delayed
import pyttsx3
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler,quantile_transform,PolynomialFeatures
from librairies.bt import *
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score, precision_recall_curve, confusion_matrix, classification_report
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax
import colorama as col
from collections import Counter
from librairies.strategy import *
scaler = MinMaxScaler(feature_range=(0, 1))
engine = pyttsx3.init()
engine.say("Librairies loaded")
engine.runAndWait() 

Importing Librairies...


###### RELOAD LIBRAIRIE #####
from importlib import reload
sys.path.append('../')
librairies.dagfeaturingfx= reload(librairies.dagfeaturingfx)
from  librairies.dagfeaturingfx import *

%%time

_t1 = dt.datetime.now()
print('Début des opérations horodatée à',col.Fore.YELLOW,dt.datetime.now(),col.Style.RESET_ALL)

TICKER_LIST = ['EUR/AUD']
x=TICKER_LIST[0]
_ticker = x.replace('/','')
_period = 'm5'
df_all = pd.read_csv(x.replace('/','')+'_'+_period+'_BidAndAsk.csv')

##### Ajout de la colonne Symbol pour identifier le ticker
df_all['Symbol'] = _ticker

##### On fixe la date en index sous forme de Timestamp
df_all['Lindex'] = pd.to_datetime(df_all['Date'] + ' ' + df_all['Time'])
df_all.set_index(pd.to_datetime(df_all.Lindex,format='%Y-%m-%d %H:%M:%S'),drop=True,inplace=True)

###### On drop les colonnes inutiles
df_all = df_all.drop(['Date','Lindex','Time','Total Ticks'],axis=1)

##### On enlève les jours correspondant au samedi et au dimanche
df_all['WE'] = np.where(((df_all.index.weekday == 5) | (df_all.index.weekday == 6)),None,df_all.index.weekday)
df_all = df_all.dropna()
df_all = df_all.drop(['WE'],axis=1)

##### Calcul des averages pour les OHLC
df_all['Open'] = (df_all['OpenBid'] + df_all['OpenAsk']) / 2
df_all['High'] = (df_all['HighBid'] + df_all['HighAsk']) / 2
df_all['Low'] = (df_all['LowBid'] + df_all['LowAsk']) / 2
df_all['Close'] = (df_all['CloseBid'] + df_all['CloseAsk']) / 2

hourly_all = pd.read_csv(x.replace('/','')+'_H1_BidAndAsk.csv')

##### Ajout de la colonne Symbol pour identifier le ticker
hourly_all['Symbol'] = _ticker

##### On fixe la date en index sous forme de Timestamp
hourly_all['Lindex'] = pd.to_datetime(hourly_all['Date'] + ' ' + hourly_all['Time'])
hourly_all.set_index(pd.to_datetime(hourly_all.Lindex,format='%Y-%m-%d %H:%M:%S'),drop=True,inplace=True)

###### On drop les colonnes inutiles
hourly_all = hourly_all.drop(['Date','Lindex','Time','Total Ticks'],axis=1)

##### On enlève les jours correspondant au samedi et au dimanche
hourly_all['WE'] = np.where(((hourly_all.index.weekday == 5) | (hourly_all.index.weekday == 6)),None,hourly_all.index.weekday)
hourly_all = hourly_all.dropna()
hourly_all = hourly_all.drop(['WE'],axis=1)

##### Calcul des averages pour les OHLC
hourly_all['Open'] = (hourly_all['OpenBid'] + hourly_all['OpenAsk']) / 2
hourly_all['High'] = (hourly_all['HighBid'] + hourly_all['HighAsk']) / 2
hourly_all['Low'] = (hourly_all['LowBid'] + hourly_all['LowAsk']) / 2
hourly_all['Close'] = (hourly_all['CloseBid'] + hourly_all['CloseAsk']) / 2

engine.say("Processing featuring of dataframes, daily and intra-day")
engine.runAndWait()
##### Récupération des data pour tous les tickers sur la période demandée en intraday => df_all

engine.say("Raw data are loaded in memory")
engine.runAndWait()


df_all = timerange1D(df_all)
hourly_all = timerange1D(hourly_all)
daily_all = get_daily(hourly_all,TICKER_LIST)

engine.say("Daily up to date")
engine.runAndWait()

daily_all = timerange1W(daily_all)
weekly_all = get_weekly(daily_all,TICKER_LIST)
engine.say("Weekly up to date")
engine.runAndWait()

##### On calcule l'ADR sur le daily
daily_all = adr(daily_all,_window=14)
engine.say("ADR computed")
engine.runAndWait()

##### On récupère l'ADR qui a été calculé en daily (daily_all) pour le mettre dans la base intraday df_all
df_all = getadr(daily_all,df_all,TICKER_LIST)
engine.say("ADR get in daily")
engine.runAndWait()

df_all = adrhnl(daily_all,df_all,TICKER_LIST)

##### Calcul d'une SMA 200 sur df_all
df_all = sma(df_all=df_all,_window=200)

##### Calcul des bollinger sur df_all
df_all = bollinger(df_all,_slow=20)

##### Calcul du stochastic slow. Par défaut les paramètres sont 5 et 3.
df_all = slowstochastic(df_all,TICKER_LIST)

df_all = ema(df_all,21,TICKER_LIST)

df_all = ema(df_all,8,TICKER_LIST)

weekly_all = pivot(weekly_all,TICKER_LIST)

df_all = pivotimportdf(df_all,weekly_all,TICKER_LIST)

df_all = atr(df_all,TICKER_LIST,14)

df_all = rvi(df_all,TICKER_LIST,_window=14)

df_all = sbgamma(df_all,TICKER_LIST)

df_all = onhisma(df_all,TICKER_LIST,_window=5)
df_all = onlosma(df_all,TICKER_LIST,_window=5)

df_all = onhisma(df_all,TICKER_LIST,_window=21)
df_all = onlosma(df_all,TICKER_LIST,_window=21)

df_all = onhisma(df_all,TICKER_LIST,_window=34)
df_all = onlosma(df_all,TICKER_LIST,_window=34)

df_all = importohlc(df_all,weekly_all,TICKER_LIST,_suffix='_weekly')

df_all = importohlc(df_all=df_all,other_all=daily_all,TICKER_LIST=TICKER_LIST,_suffix='_daily')

engine.say("The job is done")
engine.runAndWait()


engine.say("AAll the bases are saved")
engine.runAndWait()

_t2 = dt.datetime.now()
print('Début des opérations horodatée à',col.Fore.YELLOW,dt.datetime.now(),col.Style.RESET_ALL)
print((_t2 - _t1))

%%time
joblib.dump(df_all,x.replace('/','')+'_DF_ALL')
joblib.dump(hourly_all,x.replace('/','')+'_HOURLY_ALL')
joblib.dump(weekly_all,x.replace('/','')+'_WEEKLY_ALL')
joblib.dump(daily_all,x.replace('/','')+'_DAILY_ALL')

In [2]:
%%time
x='EUR/USD'
df_all = joblib.load(x.replace('/','')+'_DF_ALL')
hourly_all = joblib.load(x.replace('/','')+'_HOURLY_ALL')
weekly_all = joblib.load(x.replace('/','')+'_WEEKLY_ALL')
daily_all = joblib.load(x.replace('/','')+'_DAILY_ALL')


CPU times: user 257 ms, sys: 298 ms, total: 554 ms
Wall time: 580 ms


In [7]:
%%time
####### CHOIX DE LA TIME FRAME
_period = 'm5'
##### Récupération de la liste des tickers à partir du répertoire où sont rangées les bases
TICKER_LIST = ['EUR/USD']
TIK = ['JPY','CHF','CAD','GBP','AUD','NZD','SEK','NOK','MXN','ZAR','TRY','ILS','CNH','USD','HKD','0']
RATE = [105.4,0.8989,1.276,0.877,0.7676,0.7201,8.3684,8.5238,20.09,14.81,7.03,3.28,6.458,1,7.7527,1.21]
df_ratefx = pd.DataFrame(index=TIK)
df_ratefx['rate'] = RATE
#############################
##### F E A T U R I N G #####
#############################

print(df_all.shape,daily_all.shape,weekly_all.shape)
# Make our choice for the split, compliant to our backtest 
_start = '2010-01-01' # start the train there '2010-01-01'
_mid = '2018-08-29' # stop the train and begin the test there '2016-08-31'
_stop = '2019-12-30' # stop the test there. After that, it is kept for oos
_last = '2021-02-15' # '2020-12-31'
#### S T R A T E G Y

df_all = stochastic(df_all)
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[0]
_ticker = x.replace('/','')
print(x)
_year_bottom = _stop
_year_top = _last
_nb_bougie_exit = 2000000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 0
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_rate = 1/df_ratefx.loc[x[4:],'rate']
backtest = df_all[df_all.Symbol == x.replace('/','')]
_target =  0.002
_exposure = 10
_size = 200000
_sl =  0.001
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only
print()
print(col.Fore.RED,'###############################################################################################')
print(' ####################################### OOS WITHOUT AI ########################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
##### Backtest Over Night
TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.CYAN,'###############################################################################################')
print(' #################################### TRAIN/TEST WITHOUT AI ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
_year_bottom = _start
_year_top = _stop

TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.BLUE,'###############################################################################################')
print(' #################################### DENOISING & ENHANCING ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
while _nb_looser > 0 :
    
    df_all['TRACKER'] = np.where(df_all.index.isin(TRACKER),1,0)
    df_all['Valid'] = np.where(((df_all.Signal!=0)&(df_all.TRACKER==1)),1,0)
    df_all['Signal'] = np.where(((df_all.Valid==1)&(df_all.Signal==1)),1,np.where(((df_all.Valid==1)&(df_all.Signal==-1)),-1,0))
    backtest = df_all[df_all.Symbol == x.replace('/','')]
    ##### Purification of signal by denoising and enhancing
    TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
            _rate,x,_target,_exposure,_size,_sl)

df_all = joblib.load(x.replace('/','')+'_DF_ALL')
df_all = stochastic(df_all)
#features = featuring(df_all)



features = pd.DataFrame(index=df_all.index)
features['Symbol'] = df_all['Symbol']
features['Date'] = df_all['Date']
features['FEMA_21'] = np.nan_to_num((df_all['Close'] - df_all['EMA_21']).astype(np.float32)).reshape(-1, 1)
features['FEMA_8'] = np.nan_to_num((df_all['Close'] - df_all['EMA_8']).astype(np.float32)).reshape(-1, 1)
features['FADRLo'] = np.nan_to_num((df_all['Close'] - df_all['ADR_Low']).astype(np.float32)).reshape(-1, 1)
features['FADRHi'] = np.nan_to_num((df_all['Close'] - df_all['ADR_High']).astype(np.float32)).reshape(-1, 1)
features['FRVI40'] = np.nan_to_num((df_all['RVI'] - 40).astype(np.float32)).reshape(-1, 1)
features['FRVI60'] = np.nan_to_num((df_all['RVI'] - 60).astype(np.float32)).reshape(-1, 1)
features['FONLOSMA5'] = np.nan_to_num((df_all['Low'] - df_all['ONLOSMA_5']).astype(np.float32)).reshape(-1, 1)
features['FONHISMA5'] = np.nan_to_num((df_all['High'] - df_all['ONHISMA_5']).astype(np.float32)).reshape(-1, 1)
features['FONLOSMA21'] = np.nan_to_num((df_all['Low'] - df_all['ONLOSMA_21']).astype(np.float32)).reshape(-1, 1)
features['FONHISMA21'] = np.nan_to_num((df_all['High'] - df_all['ONHISMA_21']).astype(np.float32)).reshape(-1, 1)
features['FONLOSMA34'] = np.nan_to_num((df_all['Low'] - df_all['ONLOSMA_34']).astype(np.float32)).reshape(-1, 1)
features['FONHISMA34'] = np.nan_to_num((df_all['High'] - df_all['ONHISMA_34']).astype(np.float32)).reshape(-1, 1)
#features['FSBGAMMA'] = np.nan_to_num(df_all.SB_Gamma.astype(np.float32)).reshape(-1, 1)
features['FOPENWEEKLY'] = np.nan_to_num((df_all['Close'] - df_all['Open_weekly']).astype(np.float32)).reshape(-1, 1)
features['FHIGHWEEKLY'] = np.nan_to_num((df_all['Close'] - df_all['High_weekly']).astype(np.float32)).reshape(-1, 1)
features['FLOWWEEKLY'] = np.nan_to_num((df_all['Close'] - df_all['Low_weekly']).astype(np.float32)).reshape(-1, 1)
features['FCLOSEWEEKLY'] = np.nan_to_num((df_all['Close'] - df_all['Close_weekly']).astype(np.float32)).reshape(-1, 1)
features['FOPENDAILY'] = np.nan_to_num((df_all['Close'] - df_all['Open_daily']).astype(np.float32)).reshape(-1, 1)
features['FHIGHDAILY'] = np.nan_to_num((df_all['Close'] - df_all['High_daily']).astype(np.float32)).reshape(-1, 1)
features['FLOWDAILY'] = np.nan_to_num((df_all['Close'] - df_all['Low_daily']).astype(np.float32)).reshape(-1, 1)
features['FCLOSEDAILY'] = np.nan_to_num((df_all['Close'] - df_all['Close_daily']).astype(np.float32)).reshape(-1, 1)
features['FOPENHOURLY'] = np.nan_to_num((df_all['Close'] - df_all['Open_daily']).astype(np.float32)).reshape(-1, 1)
features['FHIGHHOURLY'] = np.nan_to_num((df_all['Close'] - df_all['High_daily']).astype(np.float32)).reshape(-1, 1)
features['FLOWHOURLY'] = np.nan_to_num((df_all['Close'] - df_all['Low_daily']).astype(np.float32)).reshape(-1, 1)
features['FCLOSEHOURLY'] = np.nan_to_num((df_all['Close'] - df_all['Close_daily']).astype(np.float32)).reshape(-1, 1)
features['FSMA200'] = np.nan_to_num((df_all['Close'] - df_all['SMA_200']).astype(np.float32)).reshape(-1, 1)
features['FBOLUP20'] = np.nan_to_num((df_all['Close'] - df_all['UpperBand']).astype(np.float32)).reshape(-1, 1)
features['FBOLLOW20'] = np.nan_to_num((df_all['Close'] - df_all['LowerBand']).astype(np.float32)).reshape(-1, 1)
features['FPP'] = np.nan_to_num((df_all['Close'] - df_all['PP']).astype(np.float32)).reshape(-1, 1)
features['FS38'] = np.nan_to_num((df_all['Close'] - df_all['S38']).astype(np.float32)).reshape(-1, 1)
features['FS62'] = np.nan_to_num((df_all['Close'] - df_all['S62']).astype(np.float32)).reshape(-1, 1)
features['FS100'] = np.nan_to_num((df_all['Close'] - df_all['S100']).astype(np.float32)).reshape(-1, 1)
features['FS138'] = np.nan_to_num((df_all['Close'] - df_all['S138']).astype(np.float32)).reshape(-1, 1)
features['FS162'] = np.nan_to_num((df_all['Close'] - df_all['S162']).astype(np.float32)).reshape(-1, 1)
features['FS200'] = np.nan_to_num((df_all['Close'] - df_all['S200']).astype(np.float32)).reshape(-1, 1)
features['FR38'] = np.nan_to_num((df_all['Close'] - df_all['R38']).astype(np.float32)).reshape(-1, 1)
features['FR62'] = np.nan_to_num((df_all['Close'] - df_all['R62']).astype(np.float32)).reshape(-1, 1)
features['FR100'] = np.nan_to_num((df_all['Close'] - df_all['R100']).astype(np.float32)).reshape(-1, 1)
features['FR138'] = np.nan_to_num((df_all['Close'] - df_all['R138']).astype(np.float32)).reshape(-1, 1)
features['FR162'] = np.nan_to_num((df_all['Close'] - df_all['R162']).astype(np.float32)).reshape(-1, 1)
features['FR200'] = np.nan_to_num((df_all['Close'] - df_all['R200']).astype(np.float32)).reshape(-1, 1)
features['SBATR'] = np.nan_to_num(((df_all['Close'] - df_all['Open']) / df_all['ATR_14']).astype(np.float32)).reshape(-1, 1)
features['Signal'] = np.nan_to_num((df_all['Signal']).astype(np.float32)).reshape(-1, 1)

features['TRACKER'] = np.nan_to_num(np.where(features.index.isin(TRACKER),1,0).astype(np.float32)).reshape(-1, 1)

# First, we must have an output. We'll call it 'Valid'. It wil be where Tracker & Signal are both to 1
features['Valid'] = np.nan_to_num(np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0).astype(np.float32)).reshape(-1, 1) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

# Let's isolate the ticker on which we made the test
features = features[features.Symbol==x.replace('/','')]

# And drop the nan
features = features.dropna()
##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

a = len(features)
features['FEMA_21'] = np.nan_to_num(features['FEMA_21'] / features['FEMA_21'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FEMA_8'] = np.nan_to_num(features['FEMA_8'] / features['FEMA_8'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FADRLo'] = np.nan_to_num(features['FADRLo'] / features['FADRLo'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FADRHi'] = np.nan_to_num(features['FADRHi'] / features['FADRHi'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FRVI40'] = np.nan_to_num(features['FRVI40'] / features['FRVI40'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FRVI60'] = np.nan_to_num(features['FRVI60'] / features['FRVI60'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FONLOSMA5'] = np.nan_to_num(features['FONLOSMA5'] / features['FONLOSMA5'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FONHISMA5'] = np.nan_to_num(features['FONHISMA5'] / features['FONHISMA5'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FONLOSMA21'] = np.nan_to_num(features['FONLOSMA21'] / features['FONLOSMA21'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FONHISMA21'] = np.nan_to_num(features['FONHISMA21'] / features['FONHISMA21'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FONLOSMA34'] = np.nan_to_num(features['FONLOSMA34'] / features['FONLOSMA34'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FONHISMA34'] = np.nan_to_num(features['FONHISMA34'] / features['FONHISMA34'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
#features['FSBGAMMA'] = np.nan_to_num(df_all.SB_Gamma.astype(np.float32)).reshape(-1, 1)
features['FOPENWEEKLY'] = np.nan_to_num(features['FOPENWEEKLY'] / features['FOPENWEEKLY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FHIGHWEEKLY'] = np.nan_to_num(features['FHIGHWEEKLY'] / features['FHIGHWEEKLY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FLOWWEEKLY'] = np.nan_to_num(features['FLOWWEEKLY'] / features['FLOWWEEKLY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FCLOSEWEEKLY'] = np.nan_to_num(features['FCLOSEWEEKLY'] / features['FCLOSEWEEKLY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FOPENDAILY'] = np.nan_to_num(features['FOPENDAILY'] / features['FOPENDAILY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FHIGHDAILY'] = np.nan_to_num(features['FHIGHDAILY'] / features['FHIGHDAILY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FLOWDAILY'] = np.nan_to_num(features['FLOWDAILY'] / features['FLOWDAILY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FCLOSEDAILY'] = np.nan_to_num(features['FCLOSEDAILY'] / features['FCLOSEDAILY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FOPENHOURLY'] = np.nan_to_num(features['FOPENHOURLY'] / features['FOPENHOURLY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FHIGHHOURLY'] = np.nan_to_num(features['FHIGHHOURLY'] / features['FHIGHHOURLY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FLOWHOURLY'] = np.nan_to_num(features['FLOWHOURLY'] / features['FLOWHOURLY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FCLOSEHOURLY'] = np.nan_to_num(features['FCLOSEHOURLY'] / features['FCLOSEHOURLY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FSMA200'] = np.nan_to_num(features['FSMA200'] / features['FSMA200'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FBOLUP20'] = np.nan_to_num(features['FBOLUP20'] / features['FBOLUP20'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FBOLLOW20'] = np.nan_to_num(features['FBOLLOW20'] / features['FBOLLOW20'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FPP'] = np.nan_to_num(features['FPP'] / features['FPP'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FS38'] = np.nan_to_num(features['FS38'] / features['FS38'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FS62'] = np.nan_to_num(features['FS62'] / features['FS62'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FS100'] = np.nan_to_num(features['FS100'] / features['FS100'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FS138'] = np.nan_to_num(features['FS138'] / features['FS138'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FS162'] = np.nan_to_num(features['FS162'] / features['FS162'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FS200'] = np.nan_to_num(features['FS200'] / features['FS200'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FR38'] = np.nan_to_num(features['FR38'] / features['FR38'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FR62'] = np.nan_to_num(features['FR62'] / features['FR62'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FR100'] = np.nan_to_num(features['FR100'] / features['FR100'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FR138'] = np.nan_to_num(features['FR138'] / features['FR138'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FR162'] = np.nan_to_num(features['FR162'] / features['FR162'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FR200'] = np.nan_to_num(features['FR200'] / features['FR200'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['SBATR'] = np.nan_to_num(features['SBATR'] / features['SBATR'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)


features_train = features[(features.Date>=_start)&(features.Date<=_mid)]
features_test = features[(features.Date>_mid)&(features.Date<=_stop)]
features_oos = features[(features.Date>_stop)&(features.Date <= _last)]
features_train = features_train[features_train.Signal!=0]
features_test = features_test[features_test.Signal!=0]
# Proceed an MaxAbsScaler on features
#features_train,features_test,features_oos = scaling(features,x.replace('/',''),TRACKER,_start,_mid,_stop,_last,scaler=MaxAbsScaler())
#features_train,features_test,features_oos = quantile(features_train,features_test,features_oos,quantile_transform)

from sklearn.neural_network import MLPClassifier
_name = 'MLPClassifier'
_model = MLPClassifier(hidden_layer_sizes=(300,15), activation='relu', solver='adam', alpha=0.000001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=400, shuffle=True, random_state=26, tol=0.0000001, verbose=False, warm_start=True, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.8, beta_2=0.999, epsilon=1e-08)

_model.fit(features_train.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1),features_train.Valid)
yhat = _model.predict(features_test.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1))
accu = round(accuracy_score(features_test.Valid, yhat) * 100,2)
prec = round(precision_score(features_test.Valid, yhat,pos_label=1) * 100,2)
recall = round(recall_score(features_test.Valid, yhat) * 100,2)
f1 = round(f1_score(features_test.Valid, yhat) * 100,2)


print('Signaux - Accuracy :' ,accu,'%')
print('Signaux - Precision :',prec,'%')
print('Signaux - Recall :', recall,'%')
print('Achat - F-measure: :' ,f1,'%')
print('\n')
print(classification_report(features_test.Valid, yhat, target_names=['Neg', 'Pos']))
conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
tt = confusion_matrix(features_test.Valid, yhat, labels=[1,0])    #_model.classes_)

conf_matrix.loc['vrais-réels'] = tt[0]
conf_matrix.loc['faux-réels'] = tt[1]

print(conf_matrix)
print()
print(col.Fore.BLUE,'Signaux pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
_tp = tt[0][0]
_fn = tt[0][1]
_prec = round((tt[0][0]/(tt[0][0]+tt[0][1]))*100,2)
_rec = round((tt[0][1]/(tt[0][0]+tt[0][1]))*100,2)

print('Vrais signaux trouvés    : ',tt[0][0])
print('Vrais signaux non trouvé :',tt[0][1])
print('Total des vrais signaux  :',tt[0][0]+tt[0][1])
if _prec > 69 and prec > 69 :
    print(col.Fore.GREEN)
elif _prec < 51 or prec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Precision : ',_prec,'%')
if _rec > 69 and _rec > 69 :
    print(col.Fore.GREEN)
elif _rec < 51 or _rec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Recall',_rec,'%')
print(col.Style.RESET_ALL)

df_all_oos = df_all[(df_all.Date > _stop)&(df_all.Date <= _last)]#.dropna()

df_all_oos = df_all_oos[df_all_oos.Symbol==x.replace('/','')]


df_all_oos['Valid'] = _model.predict(features_oos.drop(['Date','Symbol','Signal','TRACKER','Valid'],axis=1))


df_all_oos['Signal'] = np.where((df_all_oos.Signal==1)&(df_all_oos.Valid==1),1,np.where((df_all_oos.Signal==-1)&(df_all_oos.Valid==1),-1,0))
 
##### On backtest selon le ticker selectionné sur la période déterminée

#joblib.dump(_model,'Save_'+x.replace('/','')+'_m5.sav')

print(x)
_year_bottom = _stop
_year_top = _last

#_trigger_invers = 1
#_target = 0.017
##### Backtest Over Night
FINAL_TRACKER = bt(df_all_oos,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

(808006, 57) (2884, 7) (581, 19)
EUR/USD

 ###############################################################################################
 ####################################### OOS WITHOUT AI ########################################
 ############################################################################################### 
Librairies imported

Début des opérations horodatée à 2021-03-05 13:26:44.746158

Chargement de la nouvelle base


 Le rate du ticker EUR/USD est à  1.0 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 82009/82009 [00:04<00:00, 16585.83it/s]
 For ticker  EUR/USD 
 
Total Profit & Loss : $  -52796.0 . En  1429   transactions.
 
Winners Number : 443 
 
Loosers number : 986 
BT's execution time 0:00:11.278968
 EUR/USD  results 
 Tested Period 2019-12-30  à 2021-02-15 
 Total Number of trades 1429 
Started Cash : 200000
P&L in currency:  -52796.0$ 
P&L in %:  -26.4% 
Average trade duration 43.9
# Winners  443.0
# Winners long  209.

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [55]:
%%time
df_all = joblib.load(x.replace('/','')+'_DF_ALL')
df_all = stochastic(df_all)
check_inf(df_all.drop(['Date','Symbol'],axis=1).dropna())


checking for infinity

printing the count of infinity values
It contains 9208 infinite values
It contains 9208 infinite values

printing column name where infinity is present
SB_Gamma    SB_Gamma
dtype: object

printing len(r),len(ds),len(df)
9208 803814 803814
CPU times: user 2.1 s, sys: 817 ms, total: 2.92 s
Wall time: 3.1 s


In [63]:
%%time
df_all = joblib.load(x.replace('/','')+'_DF_ALL')
df_all = stochastic(df_all)
#features = featuring(df_all)



features = pd.DataFrame(index=df_all.index)
features['Symbol'] = df_all['Symbol']
features['Date'] = df_all['Date']
features['FEMA_21'] = (df_all['Close'] - df_all['EMA_21'])
features['FEMA_8'] = (df_all['Close'] - df_all['EMA_8'])
features['FADRLo'] = (df_all['Close'] - df_all['ADR_Low'])
features['FADRHi'] = (df_all['Close'] - df_all['ADR_High'])
features['FRVI40'] = (df_all['RVI'] - 40)
features['FRVI60'] = (df_all['RVI'] - 60)
features['FONLOSMA5'] = (df_all['Low'] - df_all['ONLOSMA_5'])
features['FONHISMA5'] = (df_all['High'] - df_all['ONHISMA_5'])
features['FONLOSMA21'] = (df_all['Low'] - df_all['ONLOSMA_21'])
features['FONHISMA21'] = (df_all['High'] - df_all['ONHISMA_21'])
features['FONLOSMA34'] = (df_all['Low'] - df_all['ONLOSMA_34'])
features['FONHISMA34'] = (df_all['High'] - df_all['ONHISMA_34'])
features['FSBGAMMA'] = df_all.SB_Gamma
features['FOPENWEEKLY'] = (df_all['Close'] - df_all['Open_weekly'])
features['FHIGHWEEKLY'] = (df_all['Close'] - df_all['High_weekly'])
features['FLOWWEEKLY'] = (df_all['Close'] - df_all['Low_weekly'])
features['FCLOSEWEEKLY'] = (df_all['Close'] - df_all['Close_weekly'])
features['FOPENDAILY'] = (df_all['Close'] - df_all['Open_daily'])
#features['FHIGHDAILY'] = (df_all['Close'] - df_all['High_daily'])
features['FLOWDAILY'] = (df_all['Close'] - df_all['Low_daily'])
features['FCLOSEDAILY'] = (df_all['Close'] - df_all['Close_daily'])
features['FOPENHOURLY'] = (df_all['Close'] - df_all['Open_daily'])
#features['FHIGHHOURLY'] = (df_all['Close'] - df_all['High_daily'])
features['FLOWHOURLY'] = (df_all['Close'] - df_all['Low_daily'])
features['FCLOSEHOURLY'] = (df_all['Close'] - df_all['Close_daily'])
features['FSMA200'] = (df_all['Close'] - df_all['SMA_200'])
features['FBOLUP20'] = (df_all['Close'] - df_all['UpperBand'])
features['FBOLLOW20'] = (df_all['Close'] - df_all['LowerBand'])
features['FPP'] = (df_all['Close'] - df_all['PP'])
features['FS38'] = (df_all['Close'] - df_all['S38'])
features['FS62'] = (df_all['Close'] - df_all['S62'])
features['FS100'] = (df_all['Close'] - df_all['S100'])
features['FS138'] = (df_all['Close'] - df_all['S138'])
features['FS162'] = (df_all['Close'] - df_all['S162'])
features['FS200'] = (df_all['Close'] - df_all['S200'])
features['FR38'] = (df_all['Close'] - df_all['R38'])
features['FR62'] = (df_all['Close'] - df_all['R62'])
features['FR100'] = (df_all['Close'] - df_all['R100'])
features['FR138'] = (df_all['Close'] - df_all['R138'])
features['FR162'] = (df_all['Close'] - df_all['R162'])
features['FR200'] = (df_all['Close'] - df_all['R200'])
features['SBATR'] = ((df_all['Close'] - df_all['Open']) / df_all['ATR_14'])
features['Signal'] = (df_all['Signal'])

features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)

# First, we must have an output. We'll call it 'Valid'. It wil be where Tracker & Signal are both to 1
features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

# Let's isolate the ticker on which we made the test
features = features[features.Symbol==x.replace('/','')]

# And drop the nan
features = features.dropna()
##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

a = len(features)
features['FEMA_21'] = np.nan_to_num(features['FEMA_21'] / abs(features['FEMA_21'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FEMA_8'] = np.nan_to_num(features['FEMA_8'] / abs(features['FEMA_8'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FADRLo'] = np.nan_to_num(features['FADRLo'] / abs(features['FADRLo'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FADRHi'] = np.nan_to_num(features['FADRHi'] / abs(features['FADRHi'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FRVI40'] = np.nan_to_num(features['FRVI40'] / abs(features['FRVI40'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FRVI60'] = np.nan_to_num(features['FRVI60'] / abs(features['FRVI60'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FONLOSMA5'] = np.nan_to_num(features['FONLOSMA5'] / abs(features['FONLOSMA5'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FONHISMA5'] = np.nan_to_num(features['FONHISMA5'] / abs(features['FONHISMA5'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FONLOSMA21'] = np.nan_to_num(features['FONLOSMA21'] / abs(features['FONLOSMA21'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FONHISMA21'] = np.nan_to_num(features['FONHISMA21'] / abs(features['FONHISMA21'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FONLOSMA34'] = np.nan_to_num(features['FONLOSMA34'] / abs(features['FONLOSMA34'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FONHISMA34'] = np.nan_to_num(features['FONHISMA34'] / abs(features['FONHISMA34'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FSBGAMMA'] = np.nan_to_num(features['FSBGAMMA'] / abs(features['FSBGAMMA'].rolling(a, min_periods=1).max()).astype(np.float32)).reshape(-1, 1)
features['FOPENWEEKLY'] = np.nan_to_num(features['FOPENWEEKLY'] / abs(features['FOPENWEEKLY'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FHIGHWEEKLY'] = np.nan_to_num(features['FHIGHWEEKLY'] / abs(features['FHIGHWEEKLY'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FLOWWEEKLY'] = np.nan_to_num(features['FLOWWEEKLY'] / abs(features['FLOWWEEKLY'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FCLOSEWEEKLY'] = np.nan_to_num(features['FCLOSEWEEKLY'] / abs(features['FCLOSEWEEKLY'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FOPENDAILY'] = np.nan_to_num(features['FOPENDAILY'] / abs(features['FOPENDAILY'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
#features['FHIGHDAILY'] = np.nan_to_num(features['FHIGHDAILY'] / features['FHIGHDAILY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FLOWDAILY'] = np.nan_to_num(features['FLOWDAILY'] / abs(features['FLOWDAILY'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FCLOSEDAILY'] = np.nan_to_num(features['FCLOSEDAILY'] / abs(features['FCLOSEDAILY'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FOPENHOURLY'] = np.nan_to_num(features['FOPENHOURLY'] / abs(features['FOPENHOURLY'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
#features['FHIGHHOURLY'] = np.nan_to_num(features['FHIGHHOURLY'] / features['FHIGHHOURLY'].rolling(a, min_periods=1).max()).astype(np.float32).reshape(-1, 1)
features['FLOWHOURLY'] = np.nan_to_num(features['FLOWHOURLY'] / abs(features['FLOWHOURLY'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FCLOSEHOURLY'] = np.nan_to_num(features['FCLOSEHOURLY'] / abs(features['FCLOSEHOURLY'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FSMA200'] = np.nan_to_num(features['FSMA200'] / abs(features['FSMA200'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FBOLUP20'] = np.nan_to_num(features['FBOLUP20'] / abs(features['FBOLUP20'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FBOLLOW20'] = np.nan_to_num(features['FBOLLOW20'] / abs(features['FBOLLOW20'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FPP'] = np.nan_to_num(features['FPP'] / abs(features['FPP'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FS38'] = np.nan_to_num(features['FS38'] / abs(features['FS38'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FS62'] = np.nan_to_num(features['FS62'] / abs(features['FS62'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FS100'] = np.nan_to_num(features['FS100'] / abs(features['FS100'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FS138'] = np.nan_to_num(features['FS138'] / abs(features['FS138'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FS162'] = np.nan_to_num(features['FS162'] / abs(features['FS162'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FS200'] = np.nan_to_num(features['FS200'] / abs(features['FS200'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FR38'] = np.nan_to_num(features['FR38'] / abs(features['FR38'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FR62'] = np.nan_to_num(features['FR62'] / abs(features['FR62'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FR100'] = np.nan_to_num(features['FR100'] / abs(features['FR100'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FR138'] = np.nan_to_num(features['FR138'] / abs(features['FR138'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FR162'] = np.nan_to_num(features['FR162'] / abs(features['FR162'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['FR200'] = np.nan_to_num(features['FR200'] / abs(features['FR200'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)
features['SBATR'] = np.nan_to_num(features['SBATR'] / abs(features['SBATR'].rolling(a, min_periods=1).max())).astype(np.float32).reshape(-1, 1)


CPU times: user 4.57 s, sys: 2.56 s, total: 7.13 s
Wall time: 6.6 s


In [65]:
check_nan(features.drop(['Date','Symbol','TRACKER'],axis=1))


checking for NaN

printing the count of NaN values
It contains 0 NaN values
It contains 0 NaN values

printing column name where NaN is present
Series([], dtype: object)

printing len(r),len(ds),len(df)
0 803814 803814


In [66]:
check_inf(features.drop(['Date','Symbol','TRACKER'],axis=1))


checking for infinity

printing the count of infinity values
It contains 0 infinite values
It contains 0 infinite values

printing column name where infinity is present
Series([], dtype: object)

printing len(r),len(ds),len(df)
0 803814 803814


In [70]:
%%time
####### CHOIX DE LA TIME FRAME
_period = 'm5'
##### Récupération de la liste des tickers à partir du répertoire où sont rangées les bases
TICKER_LIST = ['EUR/USD']
TIK = ['JPY','CHF','CAD','GBP','AUD','NZD','SEK','NOK','MXN','ZAR','TRY','ILS','CNH','USD','HKD','0']
RATE = [105.4,0.8989,1.276,0.877,0.7676,0.7201,8.3684,8.5238,20.09,14.81,7.03,3.28,6.458,1,7.7527,1.21]
df_ratefx = pd.DataFrame(index=TIK)
df_ratefx['rate'] = RATE
#############################
##### F E A T U R I N G #####
#############################

print(df_all.shape,daily_all.shape,weekly_all.shape)
# Make our choice for the split, compliant to our backtest 
_start = '2010-01-01' # start the train there '2010-01-01'
_mid = '2018-08-29' # stop the train and begin the test there '2016-08-31'
_stop = '2019-12-30' # stop the test there. After that, it is kept for oos
_last = '2021-02-15' # '2020-12-31'
#### S T R A T E G Y

df_all = stochastic(df_all)
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[0]
_ticker = x.replace('/','')
print(x)
_year_bottom = _stop
_year_top = _last
_nb_bougie_exit = 2000000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 0
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_rate = 1/df_ratefx.loc[x[4:],'rate']
backtest = df_all[df_all.Symbol == x.replace('/','')]
_target =  0.002
_exposure = 10
_size = 200000
_sl =  0.001
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only
print()
print(col.Fore.RED,'###############################################################################################')
print(' ####################################### OOS WITHOUT AI ########################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
##### Backtest Over Night
TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.CYAN,'###############################################################################################')
print(' #################################### TRAIN/TEST WITHOUT AI ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
_year_bottom = _start
_year_top = _stop

TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.BLUE,'###############################################################################################')
print(' #################################### DENOISING & ENHANCING ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
while _nb_looser > 0 :
    
    df_all['TRACKER'] = np.where(df_all.index.isin(TRACKER),1,0)
    df_all['Valid'] = np.where(((df_all.Signal!=0)&(df_all.TRACKER==1)),1,0)
    df_all['Signal'] = np.where(((df_all.Valid==1)&(df_all.Signal==1)),1,np.where(((df_all.Valid==1)&(df_all.Signal==-1)),-1,0))
    backtest = df_all[df_all.Symbol == x.replace('/','')]
    ##### Purification of signal by denoising and enhancing
    TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
            _rate,x,_target,_exposure,_size,_sl)

df_all = stochastic(df_all)


features_train = features[(features.Date>=_start)&(features.Date<=_mid)]
features_test = features[(features.Date>_mid)&(features.Date<=_stop)]
features_oos = features[(features.Date>_stop)&(features.Date <= _last)]
features_train = features_train[features_train.Signal!=0]
features_test = features_test[features_test.Signal!=0]
# Proceed an MaxAbsScaler on features
#features_train,features_test,features_oos = scaling(features,x.replace('/',''),TRACKER,_start,_mid,_stop,_last,scaler=MaxAbsScaler())
#features_train,features_test,features_oos = quantile(features_train,features_test,features_oos,quantile_transform)

from sklearn.neural_network import MLPClassifier
_name = 'MLPClassifier'
_model = MLPClassifier(hidden_layer_sizes=(300,15), activation='relu', solver='adam', alpha=0.000001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=400, shuffle=True, random_state=26, tol=0.0000001, verbose=False, warm_start=True, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.8, beta_2=0.999, epsilon=1e-08)

_model.fit(features_train.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1),features_train.Valid)
yhat = _model.predict(features_test.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1))
accu = round(accuracy_score(features_test.Valid, yhat) * 100,2)
prec = round(precision_score(features_test.Valid, yhat,pos_label=1) * 100,2)
recall = round(recall_score(features_test.Valid, yhat) * 100,2)
f1 = round(f1_score(features_test.Valid, yhat) * 100,2)


print('Signaux - Accuracy :' ,accu,'%')
print('Signaux - Precision :',prec,'%')
print('Signaux - Recall :', recall,'%')
print('Achat - F-measure: :' ,f1,'%')
print('\n')
print(classification_report(features_test.Valid, yhat, target_names=['Neg', 'Pos']))
conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
tt = confusion_matrix(features_test.Valid, yhat, labels=[1,0])    #_model.classes_)

conf_matrix.loc['vrais-réels'] = tt[0]
conf_matrix.loc['faux-réels'] = tt[1]

print(conf_matrix)
print()
print(col.Fore.BLUE,'Signaux pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
_tp = tt[0][0]
_fn = tt[0][1]
_prec = round((tt[0][0]/(tt[0][0]+tt[0][1]))*100,2)
_rec = round((tt[0][1]/(tt[0][0]+tt[0][1]))*100,2)

print('Vrais signaux trouvés    : ',tt[0][0])
print('Vrais signaux non trouvé :',tt[0][1])
print('Total des vrais signaux  :',tt[0][0]+tt[0][1])
if _prec > 69 and prec > 69 :
    print(col.Fore.GREEN)
elif _prec < 51 or prec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Precision : ',_prec,'%')
if _rec > 69 and _rec > 69 :
    print(col.Fore.GREEN)
elif _rec < 51 or _rec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Recall',_rec,'%')
print(col.Style.RESET_ALL)

df_all_oos = df_all[(df_all.Date > _stop)&(df_all.Date <= _last)].dropna()

df_all_oos = df_all_oos[df_all_oos.Symbol==x.replace('/','')]


df_all_oos['Valid'] = _model.predict(features_oos.drop(['Date','Symbol','Signal','TRACKER','Valid'],axis=1))


df_all_oos['Signal'] = np.where((df_all_oos.Signal==1)&(df_all_oos.Valid==1),1,np.where((df_all_oos.Signal==-1)&(df_all_oos.Valid==1),-1,0))
 
##### On backtest selon le ticker selectionné sur la période déterminée

#joblib.dump(_model,'Save_'+x.replace('/','')+'_m5.sav')

print(x)
_year_bottom = _stop
_year_top = _last

#_trigger_invers = 1
#_target = 0.017
##### Backtest Over Night
FINAL_TRACKER = bt(df_all_oos,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

(808006, 59) (2884, 7) (581, 19)
EUR/USD

 ###############################################################################################
 ####################################### OOS WITHOUT AI ########################################
 ############################################################################################### 
Librairies imported

Début des opérations horodatée à 2021-03-05 16:31:08.368860

Chargement de la nouvelle base


 Le rate du ticker EUR/USD est à  1.0 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 82009/82009 [00:07<00:00, 11197.11it/s]
 For ticker  EUR/USD 
 
Total Profit & Loss : $  -52796.0 . En  1429   transactions.
 
Winners Number : 443 
 
Loosers number : 986 
BT's execution time 0:00:13.761840
 EUR/USD  results 
 Tested Period 2019-12-30  à 2021-02-15 
 Total Number of trades 1429 
Started Cash : 200000
P&L in currency:  -52796.0$ 
P&L in %:  -26.4% 
Average trade duration 43.9
# Winners  443.0
# Winners long  209.

ValueError: max() arg is an empty sequence

In [72]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier



_model = DecisionTreeClassifier(criterion='entropy',random_state=26,splitter='best',ccp_alpha=0.00007,max_features=7,class_weight={0: 1, 1: 1000})
_model.fit(features_train.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1),features_train.Valid)
yhat = _model.predict(features_test.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1))
accu = round(accuracy_score(features_test.Valid, yhat) * 100,2)
prec = round(precision_score(features_test.Valid, yhat,pos_label=1) * 100,2)
recall = round(recall_score(features_test.Valid, yhat) * 100,2)
f1 = round(f1_score(features_test.Valid, yhat) * 100,2)


print('Signaux - Accuracy :' ,accu,'%')
print('Signaux - Precision :',prec,'%')
print('Signaux - Recall :', recall,'%')
print('Achat - F-measure: :' ,f1,'%')
print('\n')
print(classification_report(features_test.Valid, yhat, target_names=['Neg', 'Pos']))
conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
tt = confusion_matrix(features_test.Valid, yhat, labels=[1,0])    #_model.classes_)

conf_matrix.loc['vrais-réels'] = tt[0]
conf_matrix.loc['faux-réels'] = tt[1]

print(conf_matrix)
print()
print(col.Fore.BLUE,'Signaux pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
_tp = tt[0][0]
_fn = tt[0][1]
_prec = round((tt[0][0]/(tt[0][0]+tt[0][1]))*100,2)
_rec = round((tt[0][1]/(tt[0][0]+tt[0][1]))*100,2)

print('Vrais signaux trouvés    : ',tt[0][0])
print('Vrais signaux non trouvé :',tt[0][1])
print('Total des vrais signaux  :',tt[0][0]+tt[0][1])
if _prec > 69 and prec > 69 :
    print(col.Fore.GREEN)
elif _prec < 51 or prec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Precision : ',_prec,'%')
if _rec > 69 and _rec > 69 :
    print(col.Fore.GREEN)
elif _rec < 51 or _rec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Recall',_rec,'%')
print(col.Style.RESET_ALL)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [62]:
np.nan_to_num(df_all_oos[df_all_oos.index=='2020-01-02 11:40:00'].SB_Gamma.astype(np.float32)).reshape(-1, 1)

array([[0.]], dtype=float32)